In [1]:
%%capture
%run 00_Dataset.ipynb
%run 01_Data_Preparation.ipynb
%run 03_Model_RNN_BNN.ipynb

2022-04-28 09:33:01.206423: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 09:33:01.230143: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 09:33:01.230293: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 09:33:01.230788: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Input pipeline

In [2]:
# prepare data generators using tf.data
train_dataset = (
    tf.data.Dataset.from_tensor_slices((z_train,mu_train))
    #.shuffle(TRAIN_LENGTH, reshuffle_each_iteration=True)
    .batch(BATCH_SIZE, drop_remainder=True)
);

#test_dataset = tf.data.Dataset.from_tensor_slices((z_test,mu_test)).batch(BATCH_SIZE);
val_dataset = (
    tf.data.Dataset.from_tensor_slices((z_val,mu_val))
    .batch(BATCH_SIZE, drop_remainder=True)
);

# Train

In [3]:
model = my_model()

In [4]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(1e-3), 
    loss = tf.keras.losses.MeanSquaredError(), 
    metrics = tf.keras.metrics.RootMeanSquaredError())

In [ ]:
# Create a callback that saves the model's weights
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(out_dir,'cp.ckpt'), 
    monitor='val_loss', 
    save_weights_only=True, 
    verbose=1, 
    save_best_only=True
    )

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=10, verbose=1,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=1e-10
)

early_stopping = [
    tf.keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_loss",
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=0,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=500,
        verbose=1
    )
]

model_history = model.fit(train_dataset,
                          epochs = EPOCHS,
                          validation_data = val_dataset,
                          callbacks = [checkpoint, early_stopping]
                          )

Epoch 1/10000


2022-04-28 09:33:04.139098: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100


823/836 [============================>.] - ETA: 0s - loss: 131.9118 - root_mean_squared_error: 11.4853
Epoch 00001: val_loss improved from inf to 6.66850, saving model to ../out/pantheon/cp.ckpt
836/836 [==============================] - 5s 3ms/step - loss: 130.1012 - root_mean_squared_error: 11.4062 - val_loss: 6.6685 - val_root_mean_squared_error: 2.5823
Epoch 2/10000
833/836 [============================>.] - ETA: 0s - loss: 11.6063 - root_mean_squared_error: 3.4068
Epoch 00002: val_loss improved from 6.66850 to 6.45114, saving model to ../out/pantheon/cp.ckpt
836/836 [==============================] - 2s 3ms/step - loss: 11.6025 - root_mean_squared_error: 3.4062 - val_loss: 6.4511 - val_root_mean_squared_error: 2.5399
Epoch 3/10000
829/836 [============================>.] - ETA: 0s - loss: 11.2913 - root_mean_squared_error: 3.3603
Epoch 00003: val_loss improved from 6.45114 to 6.33311, saving model to ../out/pantheon/cp.ckpt
836/836 [==============================] - 2s 3ms/step - 

830/836 [============================>.] - ETA: 0s - loss: 5.0295 - root_mean_squared_error: 2.2426
Epoch 00025: val_loss did not improve from 0.32022
836/836 [==============================] - 3s 3ms/step - loss: 5.0237 - root_mean_squared_error: 2.2414 - val_loss: 0.4465 - val_root_mean_squared_error: 0.6682
Epoch 26/10000
819/836 [============================>.] - ETA: 0s - loss: 4.7358 - root_mean_squared_error: 2.1762
Epoch 00026: val_loss did not improve from 0.32022
836/836 [==============================] - 3s 3ms/step - loss: 4.6993 - root_mean_squared_error: 2.1678 - val_loss: 0.6692 - val_root_mean_squared_error: 0.8181
Epoch 27/10000
823/836 [============================>.] - ETA: 0s - loss: 5.0118 - root_mean_squared_error: 2.2387
Epoch 00027: val_loss did not improve from 0.32022
836/836 [==============================] - 2s 3ms/step - loss: 4.9856 - root_mean_squared_error: 2.2328 - val_loss: 0.3887 - val_root_mean_squared_error: 0.6235
Epoch 28/10000
818/836 [==========

Epoch 50/10000
827/836 [============================>.] - ETA: 0s - loss: 4.2877 - root_mean_squared_error: 2.0707
Epoch 00050: val_loss did not improve from 0.16045
836/836 [==============================] - 2s 2ms/step - loss: 4.2827 - root_mean_squared_error: 2.0695 - val_loss: 0.4875 - val_root_mean_squared_error: 0.6982
Epoch 51/10000
836/836 [==============================] - ETA: 0s - loss: 4.1220 - root_mean_squared_error: 2.0303
Epoch 00051: val_loss did not improve from 0.16045
836/836 [==============================] - 2s 2ms/step - loss: 4.1220 - root_mean_squared_error: 2.0303 - val_loss: 0.4115 - val_root_mean_squared_error: 0.6415
Epoch 52/10000
836/836 [==============================] - ETA: 0s - loss: 4.3507 - root_mean_squared_error: 2.0858
Epoch 00052: val_loss did not improve from 0.16045
836/836 [==============================] - 2s 2ms/step - loss: 4.3507 - root_mean_squared_error: 2.0858 - val_loss: 0.2523 - val_root_mean_squared_error: 0.5023
Epoch 53/10000
836/

...

Epoch 975/10000
826/836 [============================>.] - ETA: 0s - loss: 0.1438 - root_mean_squared_error: 0.3792
Epoch 00975: val_loss did not improve from 0.02721
836/836 [==============================] - 3s 3ms/step - loss: 0.1437 - root_mean_squared_error: 0.3791 - val_loss: 0.0452 - val_root_mean_squared_error: 0.2127
Epoch 976/10000
835/836 [============================>.] - ETA: 0s - loss: 0.1678 - root_mean_squared_error: 0.4096
Epoch 00976: val_loss did not improve from 0.02721
836/836 [==============================] - 3s 3ms/step - loss: 0.1677 - root_mean_squared_error: 0.4095 - val_loss: 0.0431 - val_root_mean_squared_error: 0.2077
Epoch 977/10000
836/836 [==============================] - ETA: 0s - loss: 0.1384 - root_mean_squared_error: 0.3720
Epoch 00977: val_loss did not improve from 0.02721
836/836 [==============================] - 3s 3ms/step - loss: 0.1384 - root_mean_squared_error: 0.3720 - val_loss: 0.0278 - val_root_mean_squared_error: 0.1667
Epoch 978/10000


Epoch 1000/10000
825/836 [============================>.] - ETA: 0s - loss: 0.1385 - root_mean_squared_error: 0.3722
Epoch 01000: val_loss did not improve from 0.02721
836/836 [==============================] - 3s 3ms/step - loss: 0.1383 - root_mean_squared_error: 0.3719 - val_loss: 0.0512 - val_root_mean_squared_error: 0.2263
Epoch 1001/10000
822/836 [============================>.] - ETA: 0s - loss: 0.1231 - root_mean_squared_error: 0.3509
Epoch 01001: val_loss did not improve from 0.02721
836/836 [==============================] - 3s 3ms/step - loss: 0.1241 - root_mean_squared_error: 0.3523 - val_loss: 0.0449 - val_root_mean_squared_error: 0.2118
Epoch 1002/10000
831/836 [============================>.] - ETA: 0s - loss: 0.1200 - root_mean_squared_error: 0.3464
Epoch 01002: val_loss did not improve from 0.02721
836/836 [==============================] - 3s 3ms/step - loss: 0.1209 - root_mean_squared_error: 0.3477 - val_loss: 0.0556 - val_root_mean_squared_error: 0.2359
Epoch 1003/10

Epoch 1025/10000
818/836 [============================>.] - ETA: 0s - loss: 0.1149 - root_mean_squared_error: 0.3390
Epoch 01025: val_loss did not improve from 0.02721
836/836 [==============================] - 3s 3ms/step - loss: 0.1160 - root_mean_squared_error: 0.3405 - val_loss: 0.0920 - val_root_mean_squared_error: 0.3033
Epoch 1026/10000
831/836 [============================>.] - ETA: 0s - loss: 0.1172 - root_mean_squared_error: 0.3424
Epoch 01026: val_loss did not improve from 0.02721
836/836 [==============================] - 3s 3ms/step - loss: 0.1177 - root_mean_squared_error: 0.3430 - val_loss: 0.0586 - val_root_mean_squared_error: 0.2420
Epoch 1027/10000
830/836 [============================>.] - ETA: 0s - loss: 0.1190 - root_mean_squared_error: 0.3450
Epoch 01027: val_loss did not improve from 0.02721
836/836 [==============================] - 3s 3ms/step - loss: 0.1191 - root_mean_squared_error: 0.3451 - val_loss: 0.0647 - val_root_mean_squared_error: 0.2544
Epoch 1028/10

Epoch 1050/10000
834/836 [============================>.] - ETA: 0s - loss: 0.1299 - root_mean_squared_error: 0.3604
Epoch 01050: val_loss did not improve from 0.02721
836/836 [==============================] - 2s 3ms/step - loss: 0.1296 - root_mean_squared_error: 0.3600 - val_loss: 0.0486 - val_root_mean_squared_error: 0.2205
Epoch 1051/10000
834/836 [============================>.] - ETA: 0s - loss: 0.1267 - root_mean_squared_error: 0.3560
Epoch 01051: val_loss did not improve from 0.02721
836/836 [==============================] - 3s 3ms/step - loss: 0.1266 - root_mean_squared_error: 0.3558 - val_loss: 0.0556 - val_root_mean_squared_error: 0.2359
Epoch 1052/10000
829/836 [============================>.] - ETA: 0s - loss: 0.1155 - root_mean_squared_error: 0.3398
Epoch 01052: val_loss did not improve from 0.02721
836/836 [==============================] - 3s 3ms/step - loss: 0.1152 - root_mean_squared_error: 0.3393 - val_loss: 0.0368 - val_root_mean_squared_error: 0.1919
Epoch 1053/10

Epoch 1075/10000
820/836 [============================>.] - ETA: 0s - loss: 0.1111 - root_mean_squared_error: 0.3333
Epoch 01075: val_loss did not improve from 0.02721
836/836 [==============================] - 2s 3ms/step - loss: 0.1109 - root_mean_squared_error: 0.3330 - val_loss: 0.0453 - val_root_mean_squared_error: 0.2129
Epoch 1076/10000
829/836 [============================>.] - ETA: 0s - loss: 0.1185 - root_mean_squared_error: 0.3442
Epoch 01076: val_loss did not improve from 0.02721
836/836 [==============================] - 2s 3ms/step - loss: 0.1182 - root_mean_squared_error: 0.3438 - val_loss: 0.0417 - val_root_mean_squared_error: 0.2043
Epoch 1077/10000
827/836 [============================>.] - ETA: 0s - loss: 0.1211 - root_mean_squared_error: 0.3480
Epoch 01077: val_loss did not improve from 0.02721
836/836 [==============================] - 2s 3ms/step - loss: 0.1222 - root_mean_squared_error: 0.3496 - val_loss: 0.0313 - val_root_mean_squared_error: 0.1769
Epoch 1078/10

In [ ]:
# load the best weights
model.load_weights(os.path.join(out_dir, 'cp.ckpt'))

In [ ]:
model.summary()

In [ ]:

plt.plot(np.arange(20,200,), model_history.history['loss'][20:200], '-', label='train_loss')
#plt.plot(np.arange(0,epochs,1), model_history.history['val_loss'], '--', label='val_loss', )
plt.xlabel('epochs')
plt.ylabel('MeanSquaredLoss')
plt.title('epoch vs loss')
plt.legend()
plt.show()


In [ ]:
# plot epoch vs loss, epoch vs RMSE
plt.figure(figsize=(15,5))

epochs = len(model_history.history['loss'])

plt.subplot(1,2,1)
plt.plot(np.arange(0,epochs,1), model_history.history['loss'], '-', label='train_loss')
#plt.plot(np.arange(0,epochs,1), model_history.history['val_loss'], '--', label='val_loss', )
plt.xlabel('epochs')
plt.ylabel('MeanSquaredLoss')
plt.title('epoch vs loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(np.arange(0,epochs,1), model_history.history['root_mean_squared_error'], '-', label='train_rmse')
#plt.plot(np.arange(0,epochs,1), model_history.history['val_root_mean_squared_error'], '--', label='val_rmse')
plt.xlabel('epochs')
plt.ylabel('RMSE')
plt.title('epoch vs RMSE')
plt.legend()
plt.savefig(os.path.join(out_dir,'05_epoch_vs_loss.png'))
plt.show()

In [ ]:
#model.evaluate(test_dataset)

# Sample Reconstruction

In [ ]:
# Reconstruct 
z_reconstruct = np.linspace(0,8.5,100)
z_reconstruct = np.expand_dims(strided_app(z_reconstruct, 4, 4), axis=-1)

In [ ]:
z_reconstruct = z_reconstruct[:20]

In [ ]:
mu_reconstruct = model.predict(z_reconstruct, batch_size=BATCH_SIZE)

In [ ]:
# plot reconstructed z vs mu
plt.plot(z_reconstruct.flatten(), mu_reconstruct.flatten());
plt.xlabel('redshift(z)')
plt.ylabel('distance modulus(mu)')
plt.title('Distance modulus vs redshift')
plt.savefig(os.path.join(out_dir,'06_sample_reconstruction.png'))
plt.show()

# Sample Reconstruction with uncertainity

In [ ]:
model_u = model_uncertainity()

In [ ]:
model_u.load_weights(os.path.join(out_dir,'cp.ckpt'))

In [ ]:
model_u.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=tf.keras.losses.MeanSquaredError(), metrics=tf.keras.metrics.RootMeanSquaredError())

In [ ]:
mu_reconstruct_uncertainity = []
n = 1000
for i in range(n):
    mu_reconstruct_uncertainity.append(model_u.predict(z_reconstruct, batch_size=BATCH_SIZE).flatten())

mu_reconstruct_uncertainity = np.array(mu_reconstruct_uncertainity)
mean = np.mean(mu_reconstruct_uncertainity, axis=0)
std_dev = np.std(mu_reconstruct_uncertainity, axis=0)

In [ ]:
plt.fill_between(z_reconstruct.flatten(), mean-2*std_dev, mean+2*std_dev,label='Reconstruction in 2 sigma')
plt.fill_between(z_reconstruct.flatten(), mean-1*std_dev, mean+1*std_dev,label='Reconstruction in 1 sigma')

plt.plot(z_reconstruct.flatten(), mean, label='Reconstruction')
plt.errorbar(z, mu, yerr=mu_err, fmt='.r', label=dataset+' Sample');
plt.xlabel('redshift(z)')
plt.ylabel('distance modulus(mu)')
plt.title('The reconstruction of distance moduli from '+dataset+' data ')
plt.legend()
plt.show()

plt.savefig(os.path.join(out_dir,'07_sample_reconstruction_with_uncertainity.png'))